In [15]:
import pandas as pd

# data to be cleaned up — only edit NEW_SPEC_LOCALITY field
aleutian_data = 'aleutian_data.csv'
aleutian_df = pd.read_csv(aleutian_data)
aleutian_df.head()

,COLLECTING_EVENT_ID,HIGHER_GEOG,CONTINENT_OCEAN,COUNTRY,STATE_PROV,COUNTY,GEOG_REMARK,GEOWKT,GEOG_AUTH_REC_ID,VALID_CATALOG_TERM_FG,SPEC_LOCALITY,LOCALITY_ATTS,LOCALITY_ID,VERBATIM_LOCALITY,NEW_SPEC_LOCALITY,to-do
0,4118,"North Pacific Ocean, Bering Sea",North Pacific Ocean,NaN,NaN,NaN,spatial data source iho_world_seas::Bering Sea,True,1148,True,Bristol Bay - Specific locality unknown,"[{""attribute_date"": ""2022-10-25"", ""attribute_type"": ""previous geography"", ""attribute_units"": null, ""attribute_value"": ""North Pacific Ocean, Bering Sea, United States, Alaska"", ""attribute_method"": null, ""attribute_remark"": ""Pre-cleanup geography value, see https://github.com/ArctosDB/arctos/issues/5138. Recommendation: review and delete this attribute if current data are accurate."", ""attribute_determiner"": ""Dusty L. McDonald""}]",1973,Bristol Bay,"Bristol Bay, no specific locality",NaN
1,4119,"North Pacific Ocean, Bering Sea",North Pacific Ocean,NaN,NaN,NaN,spatial data source iho_world_seas::Bering Sea,True,1148,True,"Bristol Bay; perhaps Cold Bay,","[{""attribute_date"": ""2022-10-25"", ""attribute_type"": ""previous geography"", ""attribute_units"": null, ""attribute_value"": ""North Pacific Ocean, Bering Sea, United States, Alaska"", ""attribute_method"": null, ""attribute_remark"": ""Pre-cleanup geography value, see https://github.com/ArctosDB/arctos/issues/5138. Recommendation: review and delete this attribute if current data are accurate."", ""attribute_determiner"": ""Dusty L. McDonald""}]",1974,specific locality unknown; perhaps Cold Bay,Bristol Bay; perhaps Cold Bay,NaN
2,4128,"North Pacific Ocean, Bering Sea",North Pacific Ocean,NaN,NaN,NaN,spatial data source iho_world_seas::Bering Sea,True,1148,True,"specific locality unknown, Aleutian Islands","[{""attribute_date"": ""2022-10-25"", ""attribute_type"": ""previous geography"", ""attribute_units"": null, ""attribute_value"": ""North Pacific Ocean, Bering Sea, United States, Alaska, Aleutian Islands"", ""attribute_method"": null, ""attribute_remark"": ""Pre-cleanup geography value, see https://github.com/ArctosDB/arctos/issues/5138. Recommendation: review and delete this attribute if current data are accurate."", ""attribute_determiner"": ""Dusty L. McDonald""}]",1981,NaN,no specific locality,NaN
3,13688,"North Pacific Ocean, Bering Sea",North Pacific Ocean,NaN,NaN,NaN,spatial data source iho_world_seas::Bering Sea,True,1148,True,"off of Hooper Bay, Alaska, Hooper Bay Quad","[{""attribute_date"": ""2022-10-25"", ""attribute_type"": ""previous geography"", ""attribute_units"": null, ""attribute_value"": ""North America, Bering Sea, United States, Alaska, Hooper Bay Quad"", ""attribute_method"": null, ""attribute_remark"": ""Pre-cleanup geography value, see https://github.com/ArctosDB/arctos/issues/5138. Recommendation: review and delete this attribute if current data are accurate."", ""attribute_determiner"": ""Dusty L. McDonald""}, {""attribute_date"": null, ""attribute_type"": ""quad"", ""attribute_units"": null, ""attribute_value"": ""Hooper Bay"", ""attribute_method"": null, ""attribute_remark"": ""Migrated from geography."", ""attribute_determiner"": null}]",7470,"off of Hooper Bay, Alaska",off of Hooper Bay,NaN
4,15644,"North Pacific Ocean, Bering Sea",North Pacific Ocean,NaN,NaN,NaN,spatial data source iho_world_seas::Bering Sea,True,1148,True,"15-20 mi W Quinhagak, on sandbar, Kuskokwim Bay Quad","[{""attribute_date"": ""2022-10-25"", ""attribute_type"": ""previous geography"", ""attribute_units"": null, ""attribute_value"": ""North America, Bering Sea, United States, Alaska, Kuskokwim Bay Quad"", ""attribute_method"": null, ""attribute_remark"": ""Pre-cleanup geography value, see https://github.com/ArctosDB/arctos/issues/5138. Recommendation: review and delete this attribute if current data are accurate."", ""attribute_determiner"": ""Dusty L. McDonald""}, {""attribute_date"": null, ""attribute_type"": ""quad"", ""attribute_units"": null, ""attribute_val

In [9]:
# set max column & row width
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

In [10]:
locality_df = aleutian_df.loc[:, ['NEW_SPEC_LOCALITY']]

In [11]:
def separate_column_values(df, column_name):

    split_columns = df[column_name].str.split(',', expand=True)
    
    # create new column names for the split columns
    new_column_names = [f'{column_name}_{i}' for i in range(split_columns.shape[1])]
    
    # assign new column names to the split columns
    split_columns.columns = new_column_names
    
    # concatenate the original DataFrame with the new split columns
    df = pd.concat([df.drop(columns=[column_name]), split_columns], axis=1)
    
    return df

In [12]:
df_separated = separate_column_values(locality_df, 'NEW_SPEC_LOCALITY')
df_separated.head(100)

,NEW_SPEC_LOCALITY_0,NEW_SPEC_LOCALITY_1,NEW_SPEC_LOCALITY_2,NEW_SPEC_LOCALITY_3,NEW_SPEC_LOCALITY_4,NEW_SPEC_LOCALITY_5,NEW_SPEC_LOCALITY_6,NEW_SPEC_LOCALITY_7,NEW_SPEC_LOCALITY_8
0,Bristol Bay,no specific locality,None,None,None,None,None,None,None
1,Bristol Bay; perhaps Cold Bay,None,None,None,None,None,None,None,None
2,no specific locality,None,None,None,None,None,None,None,None
3,off of Hooper Bay,None,None,None,None,None,None,None,None
4,15-20 mi W Quinhagak,on sandbar,None,None,None,None,None,None,None
5,near Cape Nome,None,None,None,None,None,None,None,None
6,near Nome,Norton Sound,None,None,None,None,None,None,None
7,Nome,None,None,None,None,None,None,None,None
8,Nome area,None,None,None,None,None,None,None,None
9,near Webster Lake,Saint Paul Island,Pribilof Islands,None,None,None,None,None,None


In [13]:
def consolidate_columns_to_one(df):

    def consolidate_row(row):
        # filter out None values
        filtered_values = [value for value in row if pd.notnull(value)]
        # remove duplicates
        filtered_values = list(dict.fromkeys(filtered_values))  # Maintain order while removing duplicates
        # exclude values containing "Quad" or "Islands"
        filtered_values = [value for value in filtered_values if "Quad" not in value and "Islands" not in value]
        # remove "no specific locality" if there are other values
        if "no specific locality" in filtered_values and len(filtered_values) > 1:
            filtered_values = [value for value in filtered_values if value != "no specific locality"]
        # join remaining values with commas and spaces
        return ", ".join(filtered_values)

    # apply the consolidation function to each row
    df['consolidated_localities'] = df.apply(consolidate_row, axis=1)
    
    # return df with only the new consolidated column
    return df[['consolidated_localities']]

In [14]:
consolidated_localities_df = consolidate_columns_to_one(df_separated)
consolidated_localities_df['original_localities'] = aleutian_df['NEW_SPEC_LOCALITY']
consolidated_localities_df.head(100)

/var/folders/zc/czp2q6xx6rn17hj12nh4gtgh0000gr/T/ipykernel_53792/1054794893.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consolidated_localities_df['original_localities'] = aleutian_df['NEW_SPEC_LOCALITY']


,consolidated_localities,original_localities
0,"Bristol Bay, no specific locality","Bristol Bay, no specific locality"
1,Bristol Bay; perhaps Cold Bay,Bristol Bay; perhaps Cold Bay
2,no specific locality,no specific locality
3,off of Hooper Bay,off of Hooper Bay
4,"15-20 mi W Quinhagak, on sandbar","15-20 mi W Quinhagak, on sandbar"
5,near Cape Nome,near Cape Nome
6,"near Nome, Norton Sound","near Nome, Norton Sound"
7,Nome,Nome
8,Nome area,Nome area
9,"near Webster Lake, Saint Paul Island","near Webster Lake, Saint Paul Island, Pribilof Islands"
